# Analyse Exploratoire Des Données

### Bibliothèques 

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm
import summarytools as st
import requests
import json
import os

### Datasets 

* **Admissions_patients**

In [20]:
df_admission = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_admission.head()

,id_admission,date_entree,service,type_admission,mode_entree,motif_principal,transplantation,duree_sejour_jours,age_patient,sexe_patient,departement_patient,gravite_score
0,ADM-202400000,2024-01-01,Coeur-Metabolisme,Ambulatoire,Urgences,Infarctus,False,0,56,M,Val-de-Marne (94),3
1,ADM-202400001,2024-01-01,ORPHé (Onco/Hémato),Ambulatoire,Programmé,Myélome,False,0,59,M,Autres IDF,1
2,ADM-202400002,2024-01-01,Chirurgie Neuro-sensorielle,Hospitalisation Complète,Programmé,Otite chronique,False,3,58,F,Autres IDF,1
3,ADM-202400003,2024-01-01,Chirurgie,Hospitalisation Complète,Programmé,Chirurgie viscérale,False,2,56,M,Paris (75),1
4,ADM-202400004,2024-01-01,ORPHé (Onco/Hémato),Ambulatoire,Programmé,Myélome,False,0,63,M,Val-de-Marne (94),1


In [21]:
df_admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289103 entries, 0 to 289102
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id_admission         289103 non-null  object
 1   date_entree          289103 non-null  object
 2   service              289103 non-null  object
 3   type_admission       289103 non-null  object
 4   mode_entree          289103 non-null  object
 5   motif_principal      289103 non-null  object
 6   transplantation      289103 non-null  bool  
 7   duree_sejour_jours   289103 non-null  int64 
 8   age_patient          289103 non-null  int64 
 9   sexe_patient         289103 non-null  object
 10  departement_patient  289103 non-null  object
 11  gravite_score        289103 non-null  int64 
dtypes: bool(1), int64(3), object(8)
memory usage: 24.5+ MB


#### Distrubutions des admissions

* **Equipements** 

In [22]:
df_equipement = pd.read_csv('../data/raw/equipements_poles.csv')
df_equipement.head()

,date,service,equipement,quantite_totale,en_service,disponible,taux_utilisation
0,2024-01-01,PRAGUES_(Réa/Pneumo),Respirateurs,127,80,47,0.627
1,2024-01-01,PRAGUES_(Réa/Pneumo),Moniteurs,95,90,5,0.951
2,2024-01-01,PRAGUES_(Réa/Pneumo),Perfuseurs,106,89,17,0.841
3,2024-01-01,PRAGUES_(Réa/Pneumo),Défibrillateurs,26,19,7,0.727
4,2024-01-01,PRAGUES_(Réa/Pneumo),Ventilateurs_NIV,85,72,13,0.859


* **lits & Occupations**

In [23]:
df_ocupation = pd.read_csv('../data/raw/lits_poles.csv')
df_ocupation.head()

,date,service,lits_totaux,lits_occupes,taux_occupation
0,2024-01-01,PRAGUES_(Réa/Pneumo),86,72,0.842
1,2024-01-01,Urgences_(Passage_court),114,109,0.955
2,2024-01-01,MSN_(Neuro/Psy),420,355,0.848
3,2024-01-01,Chirurgie,324,265,0.818
4,2024-01-01,Coeur_Metabolisme,210,175,0.835


* **personel**

In [24]:
df_personnel = pd.read_csv('../data/raw/personnel_poles.csv')
df_personnel.head()

,date,service,categorie,effectif_total,effectif_present,taux_absence
0,2024-01-01,MSN_(Neuro/Psy),total,767,699,0.090
1,2024-01-01,MSN_(Neuro/Psy),medecins,105,95,0.087
2,2024-01-01,MSN_(Neuro/Psy),infirmiers,285,258,0.094
3,2024-01-01,MSN_(Neuro/Psy),aides,189,171,0.092
4,2024-01-01,Chirurgie,total,595,547,0.079


* **stocks_medicaments**

In [25]:
df_stocks = pd.read_csv('../data/raw/stocks_medicaments.csv')
df_stocks.head()

,date,medicament,conso_jour,livraison,stock_fin,alerte_rupture,jours_stock
0,2024-01-01,Paracétamol_1g,4895,0,157105,False,32.1
1,2024-01-01,Antibiotiques,3652,0,25508,True,7.0
2,2024-01-01,Morphine_IV,764,0,5716,False,7.5
3,2024-01-01,Heparine,11869,0,85331,False,7.2
4,2024-01-01,Insuline,118966,0,75434,True,0.6


* **patients**

In [26]:
df_patients = pd.read_csv('../data/raw/patients_pitie_2024.csv')
df_patients.head()

,id_patient,sexe,provenance_geo,annee_naissance_approx
0,2,F,ValDeMarne,1958
1,7,F,ValDeMarne,1963
2,9,M,Paris,1952
3,10,M,AutresIDF,1969
4,28,M,AutresIDF,1968


* **diagnostic**

In [27]:
df_diagnostic = pd.read_csv('../data/raw/diagnostics_pitie_2024.csv')
df_diagnostic.head()

,id_sejour,rang_diagnostic,cim10_code,type_diagnostic,pathologie_groupe
0,993321,0,G20,principal,neuro
1,993321,1,F32,secondaire,neuro
2,928127,0,I20,principal,cardio
3,951493,0,I63,principal,cardio
4,951493,1,I25,secondaire,cardio


* **Séjour**

In [28]:
df_sejour = pd.read_csv('../data/raw/sejours_pitie_2024.csv')
df_sejour.head()

,id_sejour,id_patient,annee,site,pole,type_hospit,date_admission,duree_sejour_jours,date_sortie,mode_entree,issue_sejour,reanimation_flag,pathologie_groupe,age
0,993321,2,2024,Pitie,Chirurgie,ambulatoire,2024-06-16,3.489103,2024-06-19 11:44:18.528284,urgence,SLD,False,neuro,68
1,928127,7,2024,Pitie,CoeurMetabolisme,complete,2024-06-28,0.644123,2024-06-28 15:27:32.222797,programme,domicile,False,cardio,100
2,951493,9,2024,Pitie,PRAGUES,ambulatoire,2024-03-30,1.036166,2024-03-31 00:52:04.704342,programme,domicile,False,cardio,98
3,851117,10,2024,Pitie,Chirurgie,ambulatoire,2024-04-27,13.425592,2024-05-10 10:12:51.130643,urgence,domicile,False,geno_urinaire,49
4,900828,28,2024,Pitie,Chirurgie,complete,2024-09-13,2.092347,2024-09-15 02:12:58.767610,urgence,domicile,False,cancer,37
